# 사전 준비

In [ ]:
!pip install transformers

**경고 메시지 끄기**

In [2]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**KcELECTRA, KoGPT2 모델과 KcELECTRA 토크나이저 불러오기**

In [3]:
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")   # 토크나이저는 KcELECTRA에 사용한 토크나이저로 고정

ELECTRA_model = AutoModel.from_pretrained("beomi/KcELECTRA-base")   # 비속어 감지의 사전학습 모델로는 KcELECTRA 사용
GPT_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')   # 챗봇의 사전학습 모델로는 KoGPT2 사용

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

# 데이터 불러오기

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

In [5]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse_Detection_Chatbot/Curse-detection-data/dataset.txt", 
                           names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
chat_data = pd.read_csv('ChatBotData.csv')

chat_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 텍스트 전처리

**전처리 함수 정의**

In [ ]:
!pip install soynlp
!pip install emoji==1.7.0

In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [9]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [10]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**전처리 및 데이터 분할**

텍스트 전처리는 비속어 감지 데이터에만 적용한다. (챗봇 데이터와 달리 댓글의 성격을 띄기 떄문에 무의미한 반복의 텍스트가 존재하기 때문)

In [39]:
import pandas as pd

In [42]:
# 비속어 감지 데이터
# train : validation = 3 : 1 (5824 = 2^6 * 7 * 13)

train_curse_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

train_curse_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

In [43]:
# 챗봇 데이터
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (0, int((chat_data.shape[0]/7)*5)))
val_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (int((chat_data.shape[0]/7)*5), int((chat_data.shape[0]/7)*7)))

# 데이터 구축, 토크나이징

**비속어 감지 데이터**

In [44]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [45]:
# 토크나이징

train_input_token = tokenizer(train_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = tokenizer(val_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [46]:
# 비속어 감지 데이터셋

class CurseDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [47]:
train_curse_dataset = CurseDataset(train_input_token, train_curse_label)
val_curse_dataset = CurseDataset(val_input_token, val_curse_label)

In [48]:
# 데이터로더

train_loader = DataLoader(train_curse_dataset, shuffle=True, batch_size=8)
val_loader = DataLoader(val_curse_dataset, shuffle=True, batch_size=8)

**챗봇 데이터**